In [0]:
import torch
import time
import random
import torch.nn as nn 
import numpy as np 
from torch.autograd import Variable
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_



class MyNet(nn.Module):
    def __init__(self,layers):
        super(MyNet,self).__init__()
        self.relu=nn.ReLU()
        self.hidden=nn.ModuleList()
        for input_size,output_size in zip(layers,layers[1:]):
            self.hidden.append(nn.Linear(input_size,output_size,bias=True))
        for i in range(len(layers)-1):
            kaiming_uniform_(self.hidden[i].weight,nonlinearity='relu')
        xavier_uniform_(self.hidden[-1].weight)
        

    def forward(self,active):
        for i in range(len(self.hidden)-1):
            active=self.hidden[i](active)
            active=nn.ReLU()(active)
        active=self.hidden[-1](active)
        active=nn.Softmax(dim=1)(active)
        
        return active
    

class NNs:
    def __init__(self,dim_feature,n_class,batch_size,n_epochs,layers_size,tol,lr=0.01):
        self.dim_feature=dim_feature
        self.n_class=n_class
        self.batch_size=batch_size
        self.n_epochs=n_epochs
        self.lr=lr
        self.layers_size=layers_size
        self.X_train=[]
        self.Y_train=[]
        self.tol=tol
    
    def get_batch(self,i,batch_size):
        batch_x=self.X_train[i*batch_size:i*batch_size+batch_size]
        batch_y=self.Y_train[i*batch_size:i*batch_size+batch_size]
        return batch_x,batch_y
    
    def fit(self,X_train,Y_train):
        rd=random.sample(range(len(Y_train)),len(Y_train))
        self.X_train=np.array(X_train)[rd]
        self.Y_train=np.array(Y_train)[rd]
        self.net=MyNet(self.layers_size)
        self.criterion=nn.CrossEntropyLoss()
        self.optimizer=torch.optim.Adam(self.net.parameters(),self.lr)
        total_batch= int(np.ceil(self.X_train.shape[0]/self.batch_size))

        self.net.train()
        for epoch in range(self.n_epochs):          
            for i in range(total_batch):
                # forward pass
                batch_x,batch_y=self.get_batch(i,self.batch_size)
                articles=Variable(torch.FloatTensor(batch_x))
                labels=Variable(torch.LongTensor(batch_y))
                self.optimizer.zero_grad()
                y_pre=self.net(articles)

                #compute loss
                loss=self.criterion(y_pre,labels)

                #backward pass
                loss.backward()
                self.optimizer.step()

                if(i%20==0):
                    print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
 				              %(epoch+1, self.n_epochs, i+1, total_batch, loss.item()))
                if(loss.item()<self.tol):
                    return
    
    def predict(self,X_test,Y_test):
        Y_test=np.array(Y_test)
        articles = Variable(torch.FloatTensor(X_test))
        outputs = self.net(articles)
        pre = torch.max(outputs.data, 1).indices.numpy()
        precision=sum(pre==Y_test)/len(Y_test)
    
        return precision


In [0]:
def loaddata(pathin):
    def convert_dense_vector(text,dim):
        v=np.zeros(dim)
        x_fea=text.split()
        for x in x_fea:
            fea=x.split(':')
            v[int(fea[0])]=float(fea[1])
        return v

    with open(pathin,'r') as f:
        d_lines=f.read().splitlines()
    '''
    with open("/content/drive/My Drive/Project2/Datasets/words_idf",'r') as f:
        dim=f.read().splitlines()
    '''
    dim=20167

    X=[]
    Y=[]
    for doc in d_lines:
        fea_doc=doc.split('<<>>')
        Y.append(int(fea_doc[0]))
        X.append(convert_dense_vector(fea_doc[2],dim))
    
    return X,Y


x_train,y_train=loaddata("/content/drive/My Drive/Project2/Datasets/train_tfidf_df=3")
x_test,y_test=loaddata("/content/drive/My Drive/Project2/Datasets/test_tfidf")

In [4]:
# optimize with SGD lr=0.1
layers_size=[20167,8000,800,20]
t=time.time()
model=NNs(20167,20,100,10,layers_size=layers_size,lr=0.1,tol=0.008)
model.fit(x_train,y_train)
print("time train =", time.time()-t)

Epoch [1/10], Step [1/114], Loss: 2.9960
Epoch [1/10], Step [21/114], Loss: 2.9955
Epoch [1/10], Step [41/114], Loss: 2.9950
Epoch [1/10], Step [61/114], Loss: 2.9946
Epoch [1/10], Step [81/114], Loss: 2.9943
Epoch [1/10], Step [101/114], Loss: 2.9931
Epoch [2/10], Step [1/114], Loss: 2.9931
Epoch [2/10], Step [21/114], Loss: 2.9918
Epoch [2/10], Step [41/114], Loss: 2.9912
Epoch [2/10], Step [61/114], Loss: 2.9905
Epoch [2/10], Step [81/114], Loss: 2.9909
Epoch [2/10], Step [101/114], Loss: 2.9875
Epoch [3/10], Step [1/114], Loss: 2.9881
Epoch [3/10], Step [21/114], Loss: 2.9847
Epoch [3/10], Step [41/114], Loss: 2.9834
Epoch [3/10], Step [61/114], Loss: 2.9821
Epoch [3/10], Step [81/114], Loss: 2.9836
Epoch [3/10], Step [101/114], Loss: 2.9713
Epoch [4/10], Step [1/114], Loss: 2.9744
Epoch [4/10], Step [21/114], Loss: 2.9494
Epoch [4/10], Step [41/114], Loss: 2.9415
Epoch [4/10], Step [61/114], Loss: 2.9357
Epoch [4/10], Step [81/114], Loss: 2.9315
Epoch [4/10], Step [101/114], Loss:

In [6]:
# optimize with SGD , lr=0.1
print("test=",model.predict(x_test,y_test))
print("train=",model.predict(x_train,y_train))

test= 0.6860063728093467
train= 0.7745271345235991


In [7]:
# optimize with SGD , lr=0.5
layers_size=[20167,8000,800,20]
t=time.time()
model=NNs(20167,20,100,10,layers_size=layers_size,lr=0.5,tol=2.11)
model.fit(x_train,y_train)
print("time train =", time.time()-t)

Epoch [1/10], Step [1/114], Loss: 2.9955
Epoch [1/10], Step [21/114], Loss: 2.9933
Epoch [1/10], Step [41/114], Loss: 2.9910
Epoch [1/10], Step [61/114], Loss: 2.9856
Epoch [1/10], Step [81/114], Loss: 2.9755
Epoch [1/10], Step [101/114], Loss: 2.9222
Epoch [2/10], Step [1/114], Loss: 2.7663
Epoch [2/10], Step [21/114], Loss: 2.6693
Epoch [2/10], Step [41/114], Loss: 2.5307
Epoch [2/10], Step [61/114], Loss: 2.4965
Epoch [2/10], Step [81/114], Loss: 2.4858
Epoch [2/10], Step [101/114], Loss: 2.3475
Epoch [3/10], Step [1/114], Loss: 2.3455
Epoch [3/10], Step [21/114], Loss: 2.3399
Epoch [3/10], Step [41/114], Loss: 2.2791
Epoch [3/10], Step [61/114], Loss: 2.2583
Epoch [3/10], Step [81/114], Loss: 2.2718
Epoch [3/10], Step [101/114], Loss: 2.1708
Epoch [4/10], Step [1/114], Loss: 2.1845
Epoch [4/10], Step [21/114], Loss: 2.2136
Epoch [4/10], Step [41/114], Loss: 2.1935
Epoch [4/10], Step [61/114], Loss: 2.1421
time train = 799.633341550827


In [8]:
# optimize with SGD , lr=0.5
print("test=",model.predict(x_test,y_test))
print("train=",model.predict(x_train,y_train))

test= 0.7946096654275093
train= 0.9420187378469154


In [3]:
# optimize with SGD , lr=1
layers_size=[20167,8000,800,20]
t=time.time()
model=NNs(20167,20,100,10,layers_size=layers_size,lr=1,tol=2.1)
model.fit(x_train,y_train)
print("time train =", time.time()-t)

Epoch [1/10], Step [1/114], Loss: 2.9956
Epoch [1/10], Step [21/114], Loss: 2.9931
Epoch [1/10], Step [41/114], Loss: 2.9733
Epoch [1/10], Step [61/114], Loss: 2.8824
Epoch [1/10], Step [81/114], Loss: 2.6172
Epoch [1/10], Step [101/114], Loss: 2.5824
Epoch [2/10], Step [1/114], Loss: 2.5676
Epoch [2/10], Step [21/114], Loss: 2.4686
Epoch [2/10], Step [41/114], Loss: 2.2649
Epoch [2/10], Step [61/114], Loss: 2.4287
Epoch [2/10], Step [81/114], Loss: 2.3257
Epoch [2/10], Step [101/114], Loss: 2.3223
Epoch [3/10], Step [1/114], Loss: 2.3692
Epoch [3/10], Step [21/114], Loss: 2.2853
Epoch [3/10], Step [41/114], Loss: 2.2018
Epoch [3/10], Step [61/114], Loss: 2.2802
Epoch [3/10], Step [81/114], Loss: 2.2384
Epoch [3/10], Step [101/114], Loss: 2.2766
time train = 632.5136737823486


In [4]:
# optimize with SGD , lr=1
print("test=",model.predict(x_test,y_test))
print("train=",model.predict(x_train,y_train))

test= 0.7615507169410515
train= 0.8974721583878381


In [5]:
# optimize with SGD , lr=0.7
layers_size=[20167,8000,800,20]
t=time.time()
model=NNs(20167,20,100,10,layers_size=layers_size,lr=0.7,tol=2.1)
model.fit(x_train,y_train)
print("time train =", time.time()-t)

Epoch [1/10], Step [1/114], Loss: 2.9954
Epoch [1/10], Step [21/114], Loss: 2.9943
Epoch [1/10], Step [41/114], Loss: 2.9886
Epoch [1/10], Step [61/114], Loss: 2.9694
Epoch [1/10], Step [81/114], Loss: 2.8202
Epoch [1/10], Step [101/114], Loss: 2.7469
Epoch [2/10], Step [1/114], Loss: 2.5386
Epoch [2/10], Step [21/114], Loss: 2.5258
Epoch [2/10], Step [41/114], Loss: 2.4703
Epoch [2/10], Step [61/114], Loss: 2.4260
Epoch [2/10], Step [81/114], Loss: 2.3219
Epoch [2/10], Step [101/114], Loss: 2.2847
Epoch [3/10], Step [1/114], Loss: 2.2472
Epoch [3/10], Step [21/114], Loss: 2.3170
Epoch [3/10], Step [41/114], Loss: 2.3201
Epoch [3/10], Step [61/114], Loss: 2.2255
Epoch [3/10], Step [81/114], Loss: 2.2314
Epoch [3/10], Step [101/114], Loss: 2.1864
time train = 654.4335124492645


In [6]:
# optimize with SGD , lr=0.7
print("test=",model.predict(x_test,y_test))
print("train=",model.predict(x_train,y_train))

test= 0.77376526818906
train= 0.9259324730422486


In [3]:
# optimize with Adam, lr=0.001
layers_size=[20167,8000,800,20]
t=time.time()
model=NNs(20167,20,100,10,layers_size=layers_size,lr=0.001,tol=2.1)
model.fit(x_train,y_train)
print("time train =", time.time()-t)

Epoch [1/10], Step [1/114], Loss: 2.9959
Epoch [1/10], Step [21/114], Loss: 2.4432
Epoch [1/10], Step [41/114], Loss: 2.2913
Epoch [1/10], Step [61/114], Loss: 2.2572
Epoch [1/10], Step [81/114], Loss: 2.1559
Epoch [1/10], Step [101/114], Loss: 2.1997
Epoch [2/10], Step [1/114], Loss: 2.0987
time train = 396.2681608200073


In [4]:
# optimize with Adam , lr=0.001
print("test=",model.predict(x_test,y_test))
print("train=",model.predict(x_train,y_train))

test= 0.7942113648433351
train= 0.9469683577868128
